In [74]:
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import xgboost as xgb

import matplotlib.pylab as plt
%matplotlib inline

pd.set_option('display.max_rows', 200)
pd.set_option('display.max_columns', 200)

In [47]:
train_a = pd.read_parquet('../A/train_targets.parquet')
train_b = pd.read_parquet('../B/train_targets.parquet')
train_c = pd.read_parquet('../C/train_targets.parquet')

In [48]:
X_train_observed_a = pd.read_parquet('../A/X_train_observed.parquet')
X_train_observed_b = pd.read_parquet('../B/X_train_observed.parquet')
X_train_observed_c = pd.read_parquet('../C/X_train_observed.parquet')

Y_train_observed_a = pd.read_parquet('../A/train_targets.parquet')
Y_train_observed_b = pd.read_parquet('../B/train_targets.parquet')
Y_train_observed_c = pd.read_parquet('../C/train_targets.parquet')

X_train_estimated_a = pd.read_parquet('../A/X_train_estimated.parquet')
X_train_estimated_b = pd.read_parquet('../B/X_train_estimated.parquet')
X_train_estimated_c = pd.read_parquet('../C/X_train_estimated.parquet')

In [49]:
X_train_observed_a.describe()

,date_forecast,absolute_humidity_2m:gm3,air_density_2m:kgm3,ceiling_height_agl:m,clear_sky_energy_1h:J,clear_sky_rad:W,cloud_base_agl:m,dew_or_rime:idx,dew_point_2m:K,diffuse_rad:W,diffuse_rad_1h:J,direct_rad:W,direct_rad_1h:J,effective_cloud_cover:p,elevation:m,fresh_snow_12h:cm,fresh_snow_1h:cm,fresh_snow_24h:cm,fresh_snow_3h:cm,fresh_snow_6h:cm,is_day:idx,is_in_shadow:idx,msl_pressure:hPa,precip_5min:mm,precip_type_5min:idx,pressure_100m:hPa,pressure_50m:hPa,prob_rime:p,rain_water:kgm2,relative_humidity_1000hPa:p,sfc_pressure:hPa,snow_density:kgm3,snow_depth:cm,snow_drift:idx,snow_melt_10min:mm,snow_water:kgm2,sun_azimuth:d,sun_elevation:d,super_cooled_liquid_water:kgm2,t_1000hPa:K,total_cloud_cover:p,visibility:m,wind_speed_10m:ms,wind_speed_u_10m:ms,wind_speed_v_10m:ms,wind_speed_w_1000hPa:ms
count,118669,118669.000000,118669.000000,96422.000000,1.186690e+05,118669.000000,110603.000000,118669.000000,118669.000000,118669.000000,1.186690e+05,118669.000000,1.186690e+05,118669.000000,118669.0,118669.000000,118669.000000,118669.000000,118669.000000,118669.000000,118669.000000,118669.000000,118669.000000,118669.000000,118669.000000,118669.000000,118669.000000,118669.000000,118669.000000,118669.000000,118669.000000,2724.0,118669.000000,118669.0,118669.000000,118669.000000,118669.000000,118669.000000,118669.000000,118669.000000,118669.000000,118669.000000,118669.000000,118669.000000,118669.000000,118669.000000
mean,2021-02-09 23:30:00,6.652576,1.249064,2907.949219,5.994018e+05,166.500381,1737.142212,0.022398,276.823395,44.918282,1.617057e+05,56.611744,2.038025e+05,67.605507,6.0,0.053117,0.004418,0.106243,0.013277,0.026567,0.521391,0.511650,1009.632996,0.006302,0.091018,996.765381,1002.868225,0.187399,0.011914,74.231705,1008.998352,250.0,0.052763,0.0,0.000062,0.091566,179.629486,1.529545,0.059087,280.708557,74.401077,32446.992188,3.043197,0.756781,0.517437,0.000019
min,2019-06-02 22:00:00,0.700000,1.145000,27.799999,0.000000e+00,0.000000,27.900000,-1.000000,250.899994,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000,6.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,944.299988,0.000000,0.000000,932.299988,938.099976,0.000000,0.000000,22.700001,943.900024,250.0,0.000000,0.0,-0.000000,0.000000,0.002000,-50.004002,0.000000,258.899994,0.000000,154.300003,0.000000,-7.300000,-9.300000,-0.000000
25%,2020-04-06 22:45:00,4.400000,1.224000,1102.900024,0.000000e+00,0.000000,590.700012,0.000000,272.100006,0.000000,0.000000e+00,0.000000,0.000000e+00,43.299999,6.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1002.099976,0.000000,0.000000,989.299988,995.299988,0.000000,0.000000,65.000000,1001.400024,250.0,0.000000,0.0,0.000000,0.000000,88.487999,-15.066000,0.000000,275.899994,54.799999,15592.400391,1.700000,-1.400000,-0.700000,0.000000
50%,2021-02-09 23:30:00,6.300000,1.247000,1882.400024,3.574770e+04,8.600000,1162.599976,0.000000,277.200012,4.100000,2.481410e+04,0.000000,8.960000e+01,80.500000,6.0,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,1010.700012,0.000000,0.000000,997.900024,1003.900024,0.000000,0.000000,76.699997,1010.000000,250.0,0.000000,0.0,0.000000,0.000000,179.436996,1.202000,0.000000,280.700012,93.699997,36359.800781,2.700000,0.400000,0.500000,0.000000
75%,2021-12-16 00:15:00,8.600000,1.272000,3969.750000,1.037819e+06,288.799988,2079.600098,0.000000,282.000000,76.000000,2.743256e+05,41.799999,1.598314e+05,98.800003,6.0,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,1018.299988,0.000000,0.000000,1005.400024,1011.500000,0.000000,0.000000,85.400002,1017.599976,250.0,0.000000,0.0,0.000000,0.100000,270.640991,18.995001,0.100000,285.200012,99.900002,48084.500000,4.100000,2.700000,1.700000,0.000000
max,2022-10-21 01:00:00,17.500000,1.427000,12328.500000,3.005720e+06,836.400024,11688.700195,1.000000,293.799988,345.700012,1.205498e+06,688.700012,2.451173e+06,100.000000,6.0,15.100000,3.900000,15.100000,7.900000,14.800000,1.000000,1.000000,1044.000000,2.230000,6.000000,1

In [69]:
X_train_observed_group = X_train_observed_a.groupby(pd.Grouper(key='date_forecast', freq='1H')).mean().reset_index()
len(X_train_observed_group)

29668

In [70]:
X_train_observed_group.tail()

,date_forecast,absolute_humidity_2m:gm3,air_density_2m:kgm3,ceiling_height_agl:m,clear_sky_energy_1h:J,clear_sky_rad:W,cloud_base_agl:m,dew_or_rime:idx,dew_point_2m:K,diffuse_rad:W,diffuse_rad_1h:J,direct_rad:W,direct_rad_1h:J,effective_cloud_cover:p,elevation:m,fresh_snow_12h:cm,fresh_snow_1h:cm,fresh_snow_24h:cm,fresh_snow_3h:cm,fresh_snow_6h:cm,is_day:idx,is_in_shadow:idx,msl_pressure:hPa,precip_5min:mm,precip_type_5min:idx,pressure_100m:hPa,pressure_50m:hPa,prob_rime:p,rain_water:kgm2,relative_humidity_1000hPa:p,sfc_pressure:hPa,snow_density:kgm3,snow_depth:cm,snow_drift:idx,snow_melt_10min:mm,snow_water:kgm2,sun_azimuth:d,sun_elevation:d,super_cooled_liquid_water:kgm2,t_1000hPa:K,total_cloud_cover:p,visibility:m,wind_speed_10m:ms,wind_speed_u_10m:ms,wind_speed_v_10m:ms,wind_speed_w_1000hPa:ms
29663,2022-10-20 21:00:00,5.575,1.2840,NaN,0.0,0.0,686.900024,0.0,275.200012,0.0,0.0,0.0,0.0,16.275,6.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1019.250000,0.0,0.0,1006.375000,1012.625000,0.0,0.0,79.125000,1018.900024,NaN,0.0,0.0,0.0,0.0,329.987762,-33.860748,0.0,277.850006,19.500,39001.398438,1.675,0.250,1.675,0.0
29664,2022-10-20 22:00:00,5.350,1.2870,NaN,0.0,0.0,149.500000,0.0,274.600006,0.0,0.0,0.0,0.0,9.700,6.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1018.974976,0.0,0.0,1006.099976,1012.349976,0.0,0.0,79.974998,1018.650024,NaN,0.0,0.0,0.0,0.0,347.845490,-36.270000,0.0,277.475006,9.700,35336.726562,1.400,0.050,1.400,0.0
29665,2022-10-20 23:00:00,5.200,1.2905,NaN,0.0,0.0,NaN,0.0,274.174988,0.0,0.0,0.0,0.0,4.525,6.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1018.650024,0.0,0.0,1005.775024,1012.049988,0.0,0.0,81.199997,1018.349976,NaN,0.0,0.0,0.0,0.0,96.250496,-36.614998,0.0,277.149994,4.525,27164.625000,1.300,-0.175,1.300,0.0
29666,2022-10-21 00:00:00,5.000,1.2920,NaN,0.0,0.0,NaN,0.0,273.675018,0.0,0.0,0.0,0.0,1.975,6.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1018.224976,0.0,0.0,1005.325012,1011.625000,0.0,0.0,83.250000,1017.924988,NaN,0.0,0.0,0.0,0.0,23.331749,-35.368500,0.0,276.700012,1.975,29933.000000,1.400,-0.700,1.175,0.0
29667,2022-10-21 01:00:00,5.000,1.2920,NaN,0.0,0.0,NaN,0.0,273.600006,0.0,0.0,0.0,0.0,0.500,6.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1017.900024,0.0,0.0,1005.000000,1011.299988,0.0,0.0,84.099998,1017.599976,NaN,0.0,0.0,0.0,0.0,34.306999,-33.417000,0.0,276.399994,0.500,36795.000000,1.500,-1.100,1.000,0.0


In [104]:
id_columns = [c for c in X_train_observed_group.columns if ":idx" in c]
non_id_columns = [c for c in X_train_observed_group.columns if ":id" not in c and "date" not in c]

X_train_observed_group[id_columns] = X_train_observed_group[id_columns].astype("category")
X_train_observed_group[non_id_columns] = X_train_observed_group[non_id_columns].astype("float")

In [100]:
Y_train_observed_a.iloc[:29668].tail()

,time,pv_measurement
29663,2022-10-20 21:00:00,0.0
29664,2022-10-20 22:00:00,0.0
29665,2022-10-20 23:00:00,0.0
29666,2022-10-21 00:00:00,0.0
29667,2022-10-28 22:00:00,0.0


In [101]:
X_train, X_test, y_train, y_test = train_test_split(X_train_observed_group, Y_train_observed_a.iloc[:29668], random_state=1)

In [102]:
X_train.dtypes

date_forecast                     datetime64[us]
absolute_humidity_2m:gm3                 float64
air_density_2m:kgm3                      float64
ceiling_height_agl:m                     float64
clear_sky_energy_1h:J                    float64
clear_sky_rad:W                          float64
cloud_base_agl:m                         float64
dew_or_rime:idx                         category
dew_point_2m:K                           float64
diffuse_rad:W                            float64
diffuse_rad_1h:J                         float64
direct_rad:W                             float64
direct_rad_1h:J                          float64
effective_cloud_cover:p                  float64
elevation:m                              float64
fresh_snow_12h:cm                        float64
fresh_snow_1h:cm                         float64
fresh_snow_24h:cm                        float64
fresh_snow_3h:cm                         float64
fresh_snow_6h:cm                         float64
is_day:idx          

In [96]:
dtrain_reg = xgb.DMatrix(X_train, y_train, enable_categorical=True)
dtest_reg = xgb.DMatrix(X_test, y_test, enable_categorical=True)

ValueError: DataFrame.dtypes for data must be int, float, bool or category. When categorical type is supplied, The experimental DMatrix parameter`enable_categorical` must be set to `True`.  Invalid columns:date_forecast: datetime64[us], absolute_humidity_2m:gm3: category, dew_or_rime:idx: category, is_day:idx: category, is_in_shadow:idx: category, precip_type_5min:idx: category, relative_humidity_1000hPa:p: category, snow_drift:idx: category, super_cooled_liquid_water:kgm2: category